In [ ]:
https://github.com/ConnorJL/GPT2.git

Setup 

In [ ]:
import glob 
import os 
import time 
from multiprocessing import Pool 

import ftfy 
import numpy as np 
import tensorflow as tf 
import tqdm 

import encoder 


Define configuration

In [ ]:
# tạo một đường dẫn đến nơi chứa tệp txt 
base_dir = "/home/connor/2/newspaper" # Path to where your .txt files are located
# số lượng tối đa các tệp văn bản trong mỗi tệp tfRecords 
files_per = 175000
# và một chuỗi là tên của tệp tf.Records được tạo ra các tệp này có dạng 
# name_i.tfrecords với i là số thứ tự của tệp 
name = "openwebtext-newspaper"
# Một đường dẫn thư mục lưu trữ thư mục tfrecords đầu ra 
output_dir = "/home/connor/out"
# Một đường dẫn thư mục lưu các tập nhật ký logs. là các tệp văn bản chứa các thông tin 
# về quá trình chạy đoạn mã vd : như thời gian bắt đầu, kết thúc, số lượng tệp được xử lý, lỗi nếu có, v.v.
log_dir = "logs"
# file một danh sahs chưa các đường dẫn đến tất cả các tệp văn bản trong thư mục base_dir 
# và các thư mục con của nó . Sử dụng hàm global.global để trả về 1 danh sách các tệp 
# có đường dẫn với định dạng txt được tìm kiếm trong base_dir và nối các đường dẫn 1 cahs an toàn 
files = glob.glob(os.path.join(base_dir,"**/*.txt"))

# số lượng quy trình mã hóa sẽ chạy 
processes = 64 
# đường dẫn cho thư mục mã hóa 
encoder_path = "gs://openwebtext/stuff/encoder" # Path to encoder files
minimum_size = 25

Utilities funcion

In [ ]:
# Xây dựng phương thức định dạng kiểu giá trị tham số cho quy trình xử lý 
def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return  tf.train.Feature(int64_list=tf.train.Int64List(value=value))

# Xây dựng phương thức trả về một thuộc tinh là đối tượng tf.train.BytesList là dữ liệu 
# đươcj sử dụng trong tf.trin.Example một định dạng để lưu trữ bảng ghi nhị phân . 
def _bytes_feature(value):
    """Return a bytes_list from a string / byte."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

# Chia danh sách thành các phần nhận 2 tham số 
# là list là 1 danh sách các bản ghi và n là lát cắt với 175000 số mẫu 
def chunks(l, n):
    # khởi tạo 1 dnah sách rỗng để lưu chữ các phần sau khi cắt 
    out = []
    # duyệt qua danh sách 0 -> số lượng mẫu trong tệp TfRecords với bước nhảy n 
    for i in range (0, len(l), n):
        # Thêm vào danh sách các phần theo thứ tự i mỗi phần sẽ có số lượng từ i đến i+ n 
        # các phần tử trong danh sách đầu vào
        out.append(l[i: i + n])
    # Trả về kết quả danh sách đầu ra 
        return out 


# Kiểm tra xem log_dir có phải là một đường dẫn thư mục đã tồn tại 
if not os.path.exists(log_dir):
    # nếu không ta sử dụng os.mikdir để tạo một đường dẫn thư mục log_dir 
    # từ chuỗi đường dẫn trước đó 
    os.mikdir(log_dir)

# khởi tạo một biến là enc gán nó  = kết quả của quy trình xử lý tệp tfrecord cuối cùng 
# và được lưu trữ dưới dạng đường dẫn thư mục 
enc = encoder.get_encoder(encoder_path)

# gán biến file_chunks = danh sách các phần được chia từ tệp tf.record 
file_chunks = chunks(files, files_per)

# In ra thông báo số lượng file và số lượng tệp trong danh sách file_chunks 
print("Got {} files, divided into {} chunks.".format(str(len(files)), str(len(file_chunks))))


Create file

In [ ]:
def create_file(args):
    # gán kết quả của i và chunk  = tham só đầu vào 
    i, chunk = args 
    # gán kết quả của s  = 1 chuỗi string 
    s = name + "_" + str(i) + ".tfrecords"
    # kiểm tra xem 1 đường dẫn được nối bởi tập nhật ký và chuỗi s hiện có đang tồn tại hay không 
    if os.path.exists(os.path.join(log_dir, s)):
        return 
    
    # kiểm tra xem có tồn tại một tệp với đường dẫn mong muốn không nếu có
    # tức nó chưa hoàn thiện 
    if os.path.exists(os.path.join(output_dir, s)):
        os.remove(os.path.join(output_dir, s))

    # Khởi tạo một trường hợp biến phạm vi 
    # mục đích tạo một đối tượng write dể ghi các bản ghi vào một tệp TFRecords
    with tf.python_io.TFRecordWriter(os.path.join(output_dir, s)) as writer: 
        good_files = 0
        current = None

        # Duyệt qua danh sách các phần mẫu trong chunk 
        # mỗi phần gồm nhiều file
        for fn in chunk: 
            # mở tất cả các file trong mỗi phần đặt ở chế độ cho phép đọc và gán nó cho biến f 
            with tf.gfile.Opent(fn, "r") as f: 
                #đọc các file f và gán kết quả cho biến d 
                d = f.read()
            # Sửa chữa các lỗi ký tự unicode trong danh sách trong biếm d bằng cahs sử dụng 
            # ftfy.fix_text với tham số NFKC chỉ định chuẩn hóa unicode được áp dụng 
            d = ftfy.fix_text(d, normalization='NFKC')
            # sử dụng encode để mã hóa và biến danh sách d thành 1 danh sách số nguyên dạng int32
            data = np.array(enc.encode(d), np.int32)
            # Kiểm tra xem mảng data có độ dài nhỏ hơn minimum_size hoặc toàn bộ phần tử bằng 0 
            # hay không 
            if data.shape[0] < minimum_size  or (data == 0).all(): # nếu văn bản ngắn hơn 25 tokens hoặc nhỏ hơn all tokens = 0 , ta bỏ qua 
                continue 
            
            # khởi tạo biến hash gán nó bằng kết quả của phép tách split từ chuỗi cuối cùng 
            # từ định dnag / đầu tiên sau đó tách đôi chuỗi đó từ dấu chấm ta lấy phần đầu tiên 
            # kết quả biến hash chứa tên các file vd (file1 , file2)
            hash = fn.split("/")[-1].split(".")[0]
            # Khởi tạo một từ điển tên là feature chứa 2 cặp key và value 
            feature = {
                # với key hash có values là tên các tệp được trích xuất dưới dạng các chuỗi byte
                # bằng cách sử dụng _bytes_feature mã hóa 
                "hash" : _bytes_feature(hash.encode()),
                # Và key text có values là dữ liệu văn bản được mã hóa dưới dạng danh sách số nguyên int64  
                # được mã hóa bằng cách sử dụng _int64_feature 
                "text": _int64_feature(data)

            }

            # tạo một đối tượng TFRecords Example là một định dạng được sử dụng để lưu trữ dữ liệu 
            # cho việc huấn luyện mô hình 
            tf_example = tf.train.Example(features= tf.train.Features(feature=feature))
            # ghi đối tượng tf_example đã tạo vào đối tượng ghi đầu ra write . 
            # gọi phương thức Sẻializetostring để chuyển đổi toàn bộ đối tượng tf_example 
            # bao gồm các từ điển và dữ liệu lồng nhau thành một chuỗi biểu diễn nhị phân 
            # Điều anyf cần thiết vì tập TFRecord lưu trữ dữ liệu ở định dạng nhị phân 
            writer.writer(tf_example.SerilizeToString())
            # sau đó tăng biến đếm tệp sử lý lên 1 
            good_files += 1 

    # File Complete 
    # mở 1 file trong thư mục log_dir với tên được tạo từ biến s đặt chỉ đingj chế độ mở để 
    # ghi đè nội dung cũ 
    with open(os.path.join(log_dir,s), "w") as f: 
        # ghi một nội dung vào file đã mở gồm số lượng file sư lý thành công 
        # và tổng số file trong chunk 
        f.write("{} / {}".format(str(good_files), str(len(chunk))))

    # Mở 1 file khác trong thư mục log_dir tên là good_file chữ a chỉ định chế độ mở 
    # để thêm nội dung vào cuối file thay vì ghi đè 
    with open(os.path.join(log_dir, "good_files.log"), "a") as f : 
        # ghi một nội dung vào cuối file với i chỉ số của chunk , số good_file, 
        # tổng số file trong chunk 
        f.write("{}: {} / {}".format(str(i), str(good_files), str(len(chunk))))
    
    # trả về kết quả biến good_file 
    return good_files

# Lấy thời gian hiện tại (tính theo giây) bằng hàm time.time().
# Giá trị này được lưu vào biến start để sử dụng sau đó tính toán thời gian thực hiện.
start = time.time()
# Tạo một biến pool tiến trình xử lý bằng thư viện multiprocessing 
# biến processes xác định số tiến trình sẽ được sử dụng song song 
pool = Pool(processes = processes)
# khởi tạo một biến good để lưu trữ tổng số file đã xử lý 
good = 0

# lặp qua dnah sách các file được chia thành từng chunk   gán kết quả cho biến g 
# Sử dụng pool.imap để thực hiện đa xử lý phương thức create_file cho từng phần tử file_chunk 
# với số lượng cần sử lý len(file_chunk)
for g in tqdm(pool.imap(create_file, enumerate(file_chunks)), total=len(file_chunks)):
    # sau khi xử lý xong số lượng các  file chunk và phần phần tử trong file chunk 
    # ta cộng giá trị g vào biến good 
    good += g 

# Lấy thời gian thực sau khi xử lý kết thúc ghi vào biến end 
end = time.time()

# ghi ra mà hình thông tin thời ian bắt đầu , kết thúc trình xử lý , số lượng file  đã xử lý tốt 
# số lượng file trên tổng số 
print("Done! In {:.2f}s, {} / {} good files.".format(end-start, str(good), str(len(files))))